In [6]:
import time
from sage.rings.polynomial.hilbert import hilbert_poincare_series

# computes degree of regularity for a homogeneous ideal
def hdegree_reg(I):
    G = I.groebner_basis()
    LT = Ideal([g.lt() for g in G])
    HS = hilbert_poincare_series(LT)
    delta = HS.numerator().degree()
    l = HS.denominator().degree()
    return(delta - l + 1)


# computes F^top
def f_top(I, R):
    f_list = I.gens()
    new_ring.<h> = PolynomialRing(R, order = 'degrevlex')
    homogenized = [new_ring(poly.homogenize()) for poly in f_list]
    final = [R(poly.subs(h=0)) for poly in homogenized] 
    return(final)


# computes degree of regularity in the nonhomogeneous case
def degree_reg(I, R):
    J = ideal(f_top(I,R))
    return hdegree_reg(J)

  
def test_rowsp(f, f_list, d, ring):
    mons = get_mons(d, ring)
    M = mac_matrix(f_list, d, ring)
    done = False
    while not done:
        rowsp = M.row_space()
        to_be_added = []
        for row in M.rref().rows():
            g = 0
            for (coeff, mon) in zip(row, mons):
                g += coeff * mon
            max_test_deg = d - g.degree()
            if (g != 0) and (max_test_deg > 0):
                for mon in get_mons(max_test_deg, ring):
                    test_p = mon*g
                    test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                    if test_vec not in rowsp:
                        to_be_added.append(test_vec)
        if to_be_added == []:
            done = True
        else:
            new_rows = M.rows() + to_be_added
            M = Matrix(new_rows)
    f_vec = vector([ring(f).monomial_coefficient(mon) for mon in mons])
    return f_vec in M.row_space()


def get_mons(d, ring, leq = True):
    if leq:
        mons = []
        for i in range(d+1):
            mons += [ring({tuple(a):1}) for a in WeightedIntegerVectors(i,[1 for gen in ring.gens()])]
    else:
        mons = [ring({tuple(a):1}) for a in WeightedIntegerVectors(d,[1 for gen in ring.gens()])]
    mons.sort()
    mons.reverse()
    return mons


def deg_fall(f, f_list, ring):
    test_deg = f.degree()
    while True:
        print(test_deg, time.strftime("%H:%M:%S", time.localtime()))
        if test_rowsp(f, f_list, test_deg, ring):
            return test_deg
        test_deg += 1
        
        
def lfd(f_list, ring):
    fall_degrees = []
    I = ideal(f_list)
    B = I.groebner_basis()
    print(B)
    for f in B:
        print(f)
        d = deg_fall(f, f_list, ring)
        if d > f.degree():
            fall_degrees.append(d)
    if fall_degrees == []:
        return 0
    else:
        return max(fall_degrees)
    
    
# Compute the Macaulay matrix in degree d of a system f = [f1,...,fs]
def mac_matrix(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    return matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs])


# Takes an RREF Macaulay matrix as input and returns a list of the corresponding polynomials
def recover_polys(M, d, ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B) # Sage doesn't know how to work with the output list unless it is a "Sequence" of polynomials


# Now instead of using mac_matrix followed by recover_polys, we ask for the new polynomials directly:
def mac_basis(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    M = matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs]).rref()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B)


# Now define a function that calculates a Gröbner basis and the solving degree (per Caminata and Gorla, 2021)
def mac_grob(f,ring):
    test_deg = 0
    while True:
        B = mac_basis(f, test_deg, ring)
        print(test_deg, B, B.is_groebner(), ideal(B) == ideal(f))
        if B.is_groebner() and ideal(B) == ideal(f):
            return (test_deg,B)
        test_deg += 1
        

# Modified version to compute the solving degree per Caminata and Gorla, 2023
def sdeg(f, ring):
    d = 0
    while True:
        d_level_base = []
        mons = get_mons(d, ring)
        M = mac_matrix(f, d, ring)
        done = False
        while not done:
            rowsp = M.row_space()
            to_be_added = []
            for row in M.rref().rows():
                g = 0
                for (coeff, mon) in zip(row, mons):
                    g += coeff * mon
                max_test_deg = d - g.degree()
                if (g != 0) and (max_test_deg > 0):
                    for mon in get_mons(max_test_deg, ring):
                        test_p = mon*g
                        test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                        if test_vec not in rowsp:
                            to_be_added.append(test_vec)
            if to_be_added == []:
                done = True
                for row in M.rref().rows():
                    g = 0
                    for (coeff, mon) in zip(row, mons):
                        g += coeff * mon
                    if g != 0:
                        d_level_base.append(g)
                if d_level_base == []:
                    d_level_base = [ring(0)]
                check_1 = Sequence(d_level_base).is_groebner()
                check_2 = (ideal(d_level_base) == ideal(f))
                print(d, Sequence(d_level_base), check_1, check_2)
                if check_1 and check_2:
                    return (d, Sequence(d_level_base))
                d += 1
            else:
                new_rows = M.rows() + to_be_added
                M = Matrix(new_rows)

In [32]:
# Example 4.1
R.<x,y> = PolynomialRing(GF(11), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = x^10 - 1
I = Ideal(f1,f2,f3)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2,f3],R), '\n')
print(sdeg([f1,f2,f3],R), '\n')
print(lfd([f1,f2,f3],R), '\n')

[y^2 - 1, x + 1] 

10 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False True
3 [x^2*y - y, x*y^2 + 1, y^3 - y, x*y + y, y^2 - 1, x + 1] True True
(3, [x^2*y - y, x*y^2 + 1, y^3 - y, x*y + y, y^2 - 1, x + 1]) 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False True
3 [x^3 + 1, x^2*y - y, x*y^2 + 1, y^3 - y, x^2 - 1, x*y + y, y^2 - 1, x + 1] True True
(3, [x^3 + 1, x^2*y - y, x*y^2 + 1, y^3 - y, x^2 - 1, x*y + y, y^2 - 1, x + 1]) 

[y^2 - 1, x + 1]
y^2 - 1
2 20:05:44
x + 1
1 20:05:44
2 20:05:44
3 20:05:44
3 



In [33]:
# Continuing Example 4.1, checking regularity of F^h
S.<x,y,h> = PolynomialRing(GF(11), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
print(ideal(h_list).groebner_basis())
len(ideal(h_list)._singular_().mres(0).betti())/4

[x^10 - h^10, x*h^2 + h^3, x*y + y*h, y^2 - h^2]


11

In [10]:
# Continuing Example 4.1, checking regularity of in(F^h)
S.<x,y,h> = PolynomialRing(GF(11), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/4

11

In [11]:
# Example 4.2
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = z^6 - 1
I = Ideal(f1,f2,f3)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2,f3],R), '\n')
print(sdeg([f1,f2,f3],R), '\n')
print(lfd([f1,f2,f3],R), '\n')

[z^6 - 1, y^2 - 1, x + 1] 

7 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False False
3 [x^2*y - y, x*y^2 + 1, y^3 - y, x*y*z + y*z, y^2*z - z, x*y + y, y^2 - 1, x + 1] True False
4 [x^3*y + y, x^2*y^2 - 1, x*y^3 + y, y^4 - 1, x^2*y*z - y*z, x*y^2*z + z, y^3*z - y*z, x*y*z^2 + y*z^2, y^2*z^2 - z^2, x^2*y - y, x*y^2 + 1, y^3 - y, x*y*z + y*z, y^2*z - z, x^2 - 1, x*y + y, y^2 - 1, x*z + z, x + 1] True False
5 Polynomial Sequence with 36 Polynomials in 3 Variables True False
6 Polynomial Sequence with 61 Polynomials in 3 Variables True True
(6, Polynomial Sequence with 61 Polynomials in 3 Variables) 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False False
3 [x^3 + 1, x^2*y - y, x*y^2 + 1, y^3 - y, x^2*z - z, x*y*z + y*z, y^2*z - z, x*z^2 + z^2, x^2 - 1, x*y + y, y^2 - 1, x*z + z, x + 1] True False
4 Polynomial Sequence with 26 Polynomials in 3 Variables True False
5 Polynomial Sequence with 45 Polynomials in 3 Variables True False
6 Polynomial Sequence with 72 Pol

In [12]:
# Continuing Example 4.2, checking regularity of F^h
S.<x,y,z,h> = PolynomialRing(GF(7), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
len(ideal(h_list)._singular_().mres(0).betti())/4

8

In [13]:
# Continuing Example 4.2, checking regularity of in(F^h)
S.<x,y,h> = PolynomialRing(GF(7), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/4

8

In [14]:
# Example 4.3
R.<x,y> = PolynomialRing(GF(13), order = "degrevlex")
f1 = x^2 - 1
f2 = y^2 - 1
I = Ideal(f1,f2)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2],R), '\n')
print(sdeg([f1,f2],R), '\n')
print(lfd([f1,f2],R), '\n')

[x^2 - 1, y^2 - 1] 

3 

0 [0] True False
1 [0] True False
2 [x^2 - 1, y^2 - 1] True True
(2, [x^2 - 1, y^2 - 1]) 

0 [0] True False
1 [0] True False
2 [x^2 - 1, y^2 - 1] True True
(2, [x^2 - 1, y^2 - 1]) 

[x^2 - 1, y^2 - 1]
x^2 - 1
2 14:42:36
y^2 - 1
2 14:42:36
0 



In [15]:
# Continuing Example 4.3, checking regularity of F^h
S.<x,y,h> = PolynomialRing(GF(13), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2]]
len(ideal(h_list)._singular_().mres(0).betti())/3

3

In [16]:
# Continuing Example 4.3, checking regularity of in(F^h)
S.<x,y,h> = PolynomialRing(GF(13), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/3

3

In [45]:
# Example 4.4
R.<w,x,y,z> = PolynomialRing(GF(5), order = "degrevlex")
f1 = x^2 - 1
f2 = x*y - 1
f3 = w^5 - w
f4 = w^4*z^4 - 1
I = Ideal(f1,f2,f3,f4)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2,f3,f4],R), '\n')
print(sdeg([f1,f2,f3,f4],R), '\n')
print(lfd([f1,f2,f3,f4],R), '\n')

[w^4 - 1, z^4 - 1, y^2 - 1, x - y] 

9 

0 [0] True False
1 [0] True False
2 [x^2 - 1, x*y - 1] False False
3 [w*x^2 - w, x^3 - y, w*x*y - w, x^2*y - y, x*y^2 - y, x^2*z - z, x*y*z - z, x^2 - 1, x*y - 1, x - y] False False
4 Polynomial Sequence with 29 Polynomials in 4 Variables True False
5 Polynomial Sequence with 66 Polynomials in 4 Variables True False
6 Polynomial Sequence with 130 Polynomials in 4 Variables True False
7 Polynomial Sequence with 230 Polynomials in 4 Variables True False
8 Polynomial Sequence with 376 Polynomials in 4 Variables False True
9 Polynomial Sequence with 580 Polynomials in 4 Variables False True
10 Polynomial Sequence with 853 Polynomials in 4 Variables False True
11 Polynomial Sequence with 1206 Polynomials in 4 Variables False True
12 Polynomial Sequence with 1652 Polynomials in 4 Variables False True
13 Polynomial Sequence with 2204 Polynomials in 4 Variables False True
14 Polynomial Sequence with 2876 Polynomials in 4 Variables False True
15 Polynomi

In [46]:
# Continuing Example 4.4, checking regularity of F^h
S.<w,x,y,z,h> = PolynomialRing(GF(5), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
len(ideal(h_list)._singular_().mres(0).betti())/5

13

In [47]:
# Continuing Example 4.4, checking regularity of in(F^h)
S.<w,x,y,z,h> = PolynomialRing(GF(5), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/6

17

In [39]:
# Example 2.4
R.<w,x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2 - x
f2 = x*y - 1
f3 = w^6 - w
f4 = w^5*z^5 - 1
I = Ideal(f1,f2,f3,f4)
#print(I.groebner_basis(), '\n') # max.Grob.deg = 5
#print(degree_reg(I,R), '\n') # d_reg = 11
#print(mac_grob([f1,f2,f3,f4],R), '\n') # solve.deg > 19, this is as far as I've gotten before the kernel dies
#print(sdeg([f1,f2,f3,f4],R), '\n') # sd = 11, computation took several hours on a previous attempt, won't repeat here
#print(lfd([f1,f2,f3,f4],R), '\n') # d_F = 11, computation took several hours on a previous attempt, won't repeat here

In [40]:
# Continuing Example 2.4, checking regularity of F^h
S.<w,x,y,z,h> = PolynomialRing(QQ, order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
print(ideal(h_list).groebner_basis())
len(ideal(h_list)._singular_().mres(0).betti())/5

[z^5*h^15 - h^20, w^5*h^10 - h^15, w*z^5*h^5 - w*h^10, w^5*z^5 - h^10, w^6 - w*h^5, y*h^3 - h^4, x*h^2 - h^3, x^2 - x*h, x*y - h^2]


15

In [41]:
# Continuing Example 2.4, checking regularity of in(F^h)
S.<w,x,y,z,h> = PolynomialRing(QQ, order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/6 # reg(in(F^h)) = 20

20

In [55]:
# Example 4.1 -- genericity check
S.<x,y,h> = PolynomialRing(GF(11), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = x^10 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3]])
for prime in Fh.saturation(ideal(x,y,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (y - h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 11     False
Ideal (y + h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 11     False


In [57]:
# Example 4.2 -- genericity check
R.<x,y,z,h> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = z^6 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3]])
for prime in Fh.saturation(ideal(x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (z + 2*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 3*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - 2*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - 3*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 2*h, y + h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + h, y + h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 3*h, y + h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field 

In [61]:
# Example 4.3 -- genericity check
R.<x,y,h> = PolynomialRing(GF(13), order = "degrevlex")
f1 = x^2 - 1
f2 = y^2 - 1
Fh = ideal([f.homogenize() for f in [f1,f2]])
for prime in Fh.saturation(ideal(x,y,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (y - h, x - h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False
Ideal (y + h, x - h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False
Ideal (y - h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False
Ideal (y + h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False


In [62]:
# Example 4.4 -- genericity check
R.<w,x,y,z,h> = PolynomialRing(GF(5), order = "degrevlex")
f1 = x^2 - 1
f2 = x*y - 1
f3 = w^5 - w
f4 = w^4*z^4 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3,f4]])
for prime in Fh.saturation(ideal(w,x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (z - 2*h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z - 2*h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z - h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z - h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + 2*h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + 2*h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False


In [63]:
# Example 2.4 -- genericity check
R.<w,x,y,z,h> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2 - x
f2 = x*y - 1
f3 = w^6 - w
f4 = w^5*z^5 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3,f4]])
for prime in Fh.saturation(ideal(w,x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (y - h, x - h, w*z + z^2 + w*h + z*h + h^2, w^2 - z*h, z^3 - w*h^2) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, z^2 - w*h, w^2 + w*z + w*h + z*h + h^2) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, w*z - h^2, w^2 + z^2 + w*h + z*h + h^2, z^3 + z^2*h + w*h^2 + z*h^2 + h^3) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, w - z, z^4 + z^3*h + z^2*h^2 + z*h^3 + h^4) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (z - h, y - h, x - h, w^4 + w^3*h + w^2*h^2 + w*h^3 + h^4) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, w - h, z^4 + z^3*h + z^2*h^2 + z*h^3 + h^4) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (z - h, y - h, x - h, w - h) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Fi

In [65]:
# Example from Minko's Thesis -- genericity check
R.<x,y,z,h> = PolynomialRing(GF(7), order = 'degrevlex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
Fh = ideal([f.homogenize() for f in f_list])
for prime in Fh.saturation(ideal(x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (z + h, y - 3*h, x - 2*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - 3*h, x + 3*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 3*h, y + h, x) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y + h, x - h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - 2*h, x + 2*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + h, y - 2*h, x - 3*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y + 3*h, x - 3*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - 3*h, y + 2*h, x) of Multivariate Polynomial Ring in x, y, z, h over Finite 